In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dropout, Dense 
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from keras_preprocessing import sequence
from keras_preprocessing.text import Tokenizer

from transformers import BertTokenizer, TFBertModel

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

import os

import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
from collections import Counter

import nltk
import re
# nltk.download('punkt')

In [8]:
csv_file = "train_cleaned.csv"
data_path = os.path.join('data')
df = pd.read_csv(os.path.join(data_path, csv_file), delimiter=',').dropna()

# kata baku bahasa indonesia
kata_baku = "kamus-kata-dasar.csv"
df_baku = pd.read_csv(kata_baku, header=None).dropna()
df_baku

# normalisasi kata "slang"
kamus_normalisasi = pd.read_csv("data/slang.csv")
kata_normalisasi_dict = {}
for index, row in kamus_normalisasi.iterrows():
    if row[0] not in kata_normalisasi_dict:
        kata_normalisasi_dict[row[0]] = row[1] 

def preprocess_text(text):
    # text = text.str.replace('3', 'e')
    text = re.sub(r'3', 'e', text)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    text = re.sub(r'\d+', '', text)
    filtered_words = [word for word in text.split()]
    return ' '.join(filtered_words)

def preprocess_dataframe(df, column_name):
    df[column_name] = df[column_name].apply(lambda x: preprocess_text(x))
    return df

def normalisasi_kata(document):
    normalized_words = [kata_normalisasi_dict.get(term, term) for term in document]
    return ' '.join(normalized_words)
    # return [kata_normalisasi_dict[term] if term in kata_normalisasi_dict else term for term in document]

def word_tokenize_wrapper(text):
    return word_tokenize(text)

train_cleaned = preprocess_dataframe(df, 'text').dropna()
train_cleaned['text'] = train_cleaned['text'].apply(word_tokenize_wrapper)
train_cleaned['text'] = train_cleaned['text'].apply(normalisasi_kata)
train_cleaned.head()

csv_file = "train_cleaned_2.csv"
train_cleaned.to_csv(os.path.join(data_path, csv_file), index=False)
print("Write CSV Done.")
# train_cleaned = pd.read_csv(os.path.join(data_path, csv_file), delimiter=',')
train_cleaned.head()

Write CSV Done.


,text,label
0,kunjung prabowo resmi serah proyek bantu bersi...,Sumber Daya Alam
1,anies tepuk tangan riah rektor wajib kuliah ko...,Politik
2,benar dukung goblok dukung ridwan kamil skema ...,Demografi
3,anies sikap kritis kerja prabowo anggap tidak ...,Politik
4,anies baswedan harap polri pegang sumpah milu,Politik


In [7]:


csv_file = "train_cleaned_2.csv"
data_path = os.path.join('data')
df = pd.read_csv(os.path.join(data_path, csv_file), delimiter=',').dropna()
df.head()

,text,label
0,kunjung prabowo resmi serah proyek bantu bersi...,Sumber Daya Alam
1,anies tepuk tangan riah rektor wajib kuliah ko...,Politik
2,benar dukung goblok dukung ridwan kamil skema ...,Demografi
3,anies sikap kritis kerja prabowo anggap tidak ...,Politik
4,anies baswedan harap polri pegang sumpah milu,Politik
